# The Wright DGP

In the [iv examples notebook](http://datahub.berkeley.edu/user-redirect/interact?account=ligonteaching&repo=ARE212_Materials&branch=master&path=iv_examples.ipynb), we begin with the "Wright DGP". Let's go through this data-generating process and understand where it comes from. 

Remember that our model is given by:

$$q_D = \alpha p + u,\qquad q_S = \beta p + v,\qquad q_D = q_S$$

Imposing market clearing (the third condition), we see that:

$$\alpha p + u = \beta p + v \Rightarrow p = \frac{v-u}{\alpha-\beta}$$

Plugging this into either of the demand functions gives:

$$q = \alpha\frac{v-u}{\alpha-\beta} + u = \frac{\alpha v - \beta u}{\alpha-\beta}$$

Writing this in matrix form, we get:

$$\begin{bmatrix}u & v\end{bmatrix}\begin{bmatrix}
\frac{-\beta}{\alpha-\beta} &\frac{-1}{\alpha-\beta}  \\
\frac{\alpha}{\alpha-\beta} & \frac{1}{\alpha-\beta}\end{bmatrix} =
\begin{bmatrix} q^* & p^* \end{bmatrix}
$$

If $u$ and $v$ are now $N-$ dimentional draws, then our dimentions would be $(N \times 2)(2 \times 2) = (N \times 2)$

So now the ingredients we need are:
1. Values for $\alpha,\beta$
2. Distributions for drawing values of $u,v$
3. An instrument that is relevant for the supply shock $v$

Now we can go through the code from class. I have reordered some of the steps for expositional convenience, but otherwise the code is the same with new comments:

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import distributions as iid

In [3]:
# Step 1: Define alpha and beta, the structural parameters of the model
(alpha,beta) = (-1,2)  

# Reduced form coefficients (see above)
pi = [[-beta/(alpha - beta), -1/(alpha - beta)],
     [ alpha/(alpha - beta), 1/(alpha - beta)]]

In [4]:
# Step 2: Define distributions for v and w
sigma = {'u':1/2,'v':1/3}
mu = {'u':2,'v':-1}

# u,v assumed independent
u = iid.norm(loc=mu['u'], scale=sigma['u'])  # Demand shocks
v = iid.norm(loc=mu['v'], scale=sigma['v'])  # Supply shocks

In [5]:
# Step 3.0: Define the distribution from which our instrument will be drawn.
# NOTE: The instrument is NOT YET RELEVANT as it has no covariance with supply.
# Reduced form coefficients
w = iid.beta(1,2,loc=-iid.beta(1,2).mean()) # Centered for convenience

In [6]:
def wright_dgp(N):
    """
    Generate data consistent with Wright (1934) hog demand and supply.

    Returns a pandas dataframe with N observations on (p,q,z), where
    z is understood to be a supply shock.
    """
    
    # Arrange shocks into an Nx2 matrix (as in math above)
    U = np.c_[u.rvs(N), v.rvs(N)]
    Udf = pd.DataFrame(U,columns=['u','v']) # For future reference

    # Matrix product gives [q,p]; label by putting into df 
    # as above! This gives [q, p] (Nx2)
    df = pd.DataFrame(U@pi,columns=['q','p'])

    # Step 3.1: Make the instrument relevant! (covariance with v \neq 0)
    # Relate v and z (need not be linear)
    unobserved_shock = w.rvs(N)/10
    df['z'] = (1-unobserved_shock)*np.exp(4*Udf['v'] - unobserved_shock)
    df['Constant'] = 1

    # Include a constant term in both X & Z
    return df[['q']],df[['Constant','p']],df[['Constant','z']]

In [9]:
df_q, df_p, df_z = wright_dgp(10)
df_q

,q
0,1.065298
1,1.532264
2,0.807028
3,1.455300
4,1.191409
5,0.666152
6,1.557018
7,0.657960
8,0.769040
9,1.190859


In [10]:
df_p

,Constant,p
0,1,0.681980
1,1,1.046208
2,1,1.015502
3,1,1.043593
4,1,1.080059
5,1,1.031518
6,1,1.440361
7,1,1.060049
8,1,0.942194
9,1,0.916130


In [11]:
df_z

,Constant,z
0,1,0.285279
1,1,0.104487
2,1,0.007987
3,1,0.076604
4,1,0.019617
5,1,0.003546
6,1,0.004907
7,1,0.002882
8,1,0.011950
9,1,0.082014
